In [4]:
import json
import pandas as pd
import numpy as np
import time
import requests

In [5]:
def get_information(method, params):
    url = 'https://api.s0.os.hmny.io/'
    headers = {'Content-Type': 'application/json'}
    data = {"jsonrpc":"2.0", "method": method, "params": params, "id":1}
    r = requests.post(url, headers=headers, data = json.dumps(data))
    content = json.loads(r.content)
    return content

In [6]:
def getBlockNumber():
    method = "hmy_blockNumber"
    params = []
    num = get_information(method, params)['result']
    return int(num, 16)

In [7]:
def getAllValidatorInformation():
    method = 'hmy_getAllValidatorInformation'
    params = [-1]
    return get_information(method, params)['result'] 

In [8]:
def getAdjustment():
    method = 'hmy_getCurrentUtilityMetrics'
    params = []
    num = get_information(method, params)['result']['Adjustment']
    return float(num)

In [9]:
def getBlockSigners(blockNum):
    method = 'hmy_getBlockSigners'
    params = [blockNum]
    return get_information(method, params)['result']

In [12]:
def R9_test():
    print("Test-R9: Block reward never drops below minimum or raises above maximum block reward")
    num = int(input("enter the number of blocks you want to test: "))
    
    current_block = getBlockNumber()
    next_block = current_block + 1
    
    acc_rewards_prev = dict()
    validator_infos = getAllValidatorInformation()
    for i in validator_infos:
        if i['currently-in-committee'] == True:
            address = i['validator']['address']
            reward_accumulated = i['lifetime']['reward-accumulated']
            acc_rewards_prev[address] = reward_accumulated
            
    current_block = getBlockNumber()
    while current_block < next_block:
        current_block = getBlockNumber()
        
    # per-shard
    # default reward = 18 ONEs
    # min reward = 0, when >= 80% staked instead of 35% (of the circulating supply)
    # max reward = 32, when ~0% staked instead of 35% (of the circulating supply)
    # so, for four shards, (min, max) = (0, 128)
    min_total_reward = 0
    max_total_reward = 128e18
    
    iterations = 0
    while iterations < num:
        print("current block", current_block)
        
        # get the validator info and compute validator rewards
        acc_rewards_curr = dict()
        validator_infos = getAllValidatorInformation()
        total_reward = 0
        for i in validator_infos:
            if i['currently-in-committee'] == True:
                address = i['validator']['address']
                reward_accumulated = i['lifetime']['reward-accumulated']
                acc_rewards_curr[address] = reward_accumulated
                if address not in acc_rewards_prev:
                    continue
                reward = reward_accumulated - acc_rewards_prev[address]
                total_reward = total_reward + reward   
        
        if total_reward < min_total_reward or total_reward > max_total_reward:
            print("Test R9: Failed, block reward below minimum or above maximum", \
                "block reward", total_reward, \
                "minimum", base_block_reward)
        last_block = current_block
        current_block = getBlockNumber()
        while current_block == last_block:
            current_block = getBlockNumber()
        
        acc_rewards_prev = acc_rewards_curr
        iterations = iterations + 1
        
    print("done!")

In [13]:
R9_test()

Test-R9: Block reward never drops below minimum or raises above maximum block reward
enter the number of blocks you want to test: 1
current block 9340
done!
